In [8]:
from diffusion_model.dataset import read_csv_files

sk = read_csv_files("/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton")
s1 = read_csv_files("/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_wrist")
s2 = read_csv_files("/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_hip")

sk_files = set(sk.keys())
s1_files = set(s1.keys())
s2_files = set(s2.keys())

common = sorted(sk_files & s1_files & s2_files)
print("skeleton files:", len(sk_files))
print("sensor1 files:", len(s1_files))
print("sensor2 files:", len(s2_files))
print("common files:", len(common))
print("example common:", common[:10])

# check lengths for a few common files
for f in common[:10]:
    print(f, "T_skel", sk[f].shape[0], "T_s1", s1[f].shape[0], "T_s2", s2[f].shape[0])

[dataset] Skipped empty/unreadable file: S51A13T02.csv
[dataset] Skipped empty/unreadable file: S50A06T01.csv
[dataset] Skipped empty/unreadable file: S60A11T01.csv
[dataset] Skipped empty/unreadable file: S60A02T02.csv
[dataset] Skipped empty/unreadable file: S50A12T03.csv
[dataset] Skipped empty/unreadable file: S52A03T01.csv
[dataset] Skipped empty/unreadable file: S50A05T02.csv
[dataset] Skipped empty/unreadable file: S50A01T02.csv
[dataset] Skipped empty/unreadable file: S52A05T03.csv
[dataset] Skipped empty/unreadable file: S50A11T03.csv
[dataset] Skipped empty/unreadable file: S50A02T05.csv
[dataset] Skipped empty/unreadable file: S50A01T03.csv
[dataset] Skipped empty/unreadable file: S50A02T04.csv
[dataset] Skipped empty/unreadable file: S56A10T02.csv
[dataset] Skipped empty/unreadable file: S52A05T05.csv
[dataset] Skipped empty/unreadable file: S52A05T04.csv
[dataset] Skipped empty/unreadable file: S50A13T04.csv
[dataset] Skipped empty/unreadable file: S50A11T01.csv
[dataset] 

In [ ]:
# from /home/qsw26/smartfall/gait_loss/latent_diff_skeleton

# ---------- common paths ----------
SKEL="/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton"
RHIP="/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_hip"
LWRI="/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_wrist"

# ---------- Stage 1 (diffusion-only) ----------
python train.py \
  --stage 1 \
  --run_name proposal_stage1 \
  --skeleton_folder "$SKEL" \
  --right_hip_folder "$RHIP" \
  --left_wrist_folder "$LWRI" \
  --stage1_mode diff \
  --window 60 --stride 30 \
  --num_joints 32 --latent_dim 256 \
  --diffusion_steps 500 \
  --imu_norm zscore

# ---------- Stage 2 (Lenc = MSE(h_joint, z0), no reg head) ----------
python train.py \
  --stage 2 \
  --run_name proposal_stage2 \
  --skeleton_folder "$SKEL" \
  --right_hip_folder "$RHIP" \
  --left_wrist_folder "$LWRI" \
  --stage1_ckpt checkpoints/proposal_stage1/stage1/best.pt \
  --window 60 --stride 30 \
  --num_joints 32 --latent_dim 256 \
  --diffusion_steps 500 \
  --imu_norm zscore

# ---------- Stage 3 (proposal-faithful conditioning + cross-attn denoiser) ----------
# Recommended strict stage separation in current code: freeze IMU in stage3.
python train.py \
  --stage 3 \
  --run_name proposal_stage3 \
  --skeleton_folder "$SKEL" \
  --right_hip_folder "$RHIP" \
  --left_wrist_folder "$LWRI" \
  --stage1_ckpt checkpoints/proposal_stage1/stage1/best.pt \
  --stage2_ckpt checkpoints/proposal_stage2/stage2/best.pt \
  --stage3_cond joint \
  --freeze_imu_in_stage3 \
  --window 60 --stride 30 \
  --num_joints 32 --latent_dim 256 \
  --num_classes 2 \
  --diffusion_steps 500 \
  --imu_norm zscore


In [9]:
SKEL="/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton"
RHIP="/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_hip"
LWRI="/home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_wrist"

In [ ]:
!python train.py \
  --stage 1 \
  --run_name proposal_stage1 \
  --skeleton_folder "$SKEL" \
  --right_hip_folder "$RHIP" \
  --left_wrist_folder "$LWRI" \
  --stage1_mode diff \
  --window 60 --stride 30 \
  --num_joints 32 --latent_dim 256 \
  --diffusion_steps 500 \
  --imu_norm zscore

2026-02-28 07:03:24,551 | INFO | train | Run dir: checkpoints/proposal_stage1
2026-02-28 07:03:24,552 | INFO | train | Device: cuda
[dataset] Skipped empty/unreadable file: S51A13T02.csv
[dataset] Skipped empty/unreadable file: S50A06T01.csv
[dataset] Skipped empty/unreadable file: S60A11T01.csv
[dataset] Skipped empty/unreadable file: S60A02T02.csv
[dataset] Skipped empty/unreadable file: S50A12T03.csv
[dataset] Skipped empty/unreadable file: S52A03T01.csv
[dataset] Skipped empty/unreadable file: S50A05T02.csv
[dataset] Skipped empty/unreadable file: S50A01T02.csv
[dataset] Skipped empty/unreadable file: S52A05T03.csv
[dataset] Skipped empty/unreadable file: S50A11T03.csv
[dataset] Skipped empty/unreadable file: S50A02T05.csv
[dataset] Skipped empty/unreadable file: S50A01T03.csv
[dataset] Skipped empty/unreadable file: S50A02T04.csv
[dataset] Skipped empty/unreadable file: S56A10T02.csv
[dataset] Skipped empty/unreadable file: S52A05T05.csv
[dataset] Skipped empty/unreadable file: S5

In [ ]:
!python train.py \
  --stage 2 \
  --run_name proposal_stage2 \
  --skeleton_folder "$SKEL" \
  --right_hip_folder "$RHIP" \
  --left_wrist_folder "$LWRI" \
  --stage1_ckpt checkpoints/proposal_stage1/stage1/best.pt \
  --window 60 --stride 30 \
  --num_joints 32 --latent_dim 256 \
  --diffusion_steps 500 \
  --imu_norm zscore

2026-02-28 03:19:35,678 | INFO | train | Run dir: checkpoints/stage1_diff
2026-02-28 03:19:35,678 | INFO | train | Device: cuda
[dataset] Skipped empty/unreadable file: S51A13T02.csv
[dataset] Skipped empty/unreadable file: S50A06T01.csv
[dataset] Skipped empty/unreadable file: S60A11T01.csv
[dataset] Skipped empty/unreadable file: S60A02T02.csv
[dataset] Skipped empty/unreadable file: S50A12T03.csv
[dataset] Skipped empty/unreadable file: S52A03T01.csv
[dataset] Skipped empty/unreadable file: S50A05T02.csv
[dataset] Skipped empty/unreadable file: S50A01T02.csv
[dataset] Skipped empty/unreadable file: S52A05T03.csv
[dataset] Skipped empty/unreadable file: S50A11T03.csv
[dataset] Skipped empty/unreadable file: S50A02T05.csv
[dataset] Skipped empty/unreadable file: S50A01T03.csv
[dataset] Skipped empty/unreadable file: S50A02T04.csv
[dataset] Skipped empty/unreadable file: S56A10T02.csv
[dataset] Skipped empty/unreadable file: S52A05T05.csv
[dataset] Skipped empty/unreadable file: S52A05

In [ ]:
!python train.py \
  --stage 3 \
  --run_name proposal_stage3 \
  --skeleton_folder "$SKEL" \
  --right_hip_folder "$RHIP" \
  --left_wrist_folder "$LWRI" \
  --stage1_ckpt checkpoints/proposal_stage1/stage1/best.pt \
  --stage2_ckpt checkpoints/proposal_stage2/stage2/best.pt \
  --stage3_cond joint \
  --freeze_imu_in_stage3 \
  --window 60 --stride 30 \
  --num_joints 32 --latent_dim 256 \
  --num_classes 2 \
  --diffusion_steps 500 \
  --imu_norm zscore

2026-02-28 04:03:21,361 | INFO | train | Run dir: checkpoints/stage2_imu2latent
2026-02-28 04:03:21,361 | INFO | train | Device: cuda
[dataset] Skipped empty/unreadable file: S51A13T02.csv
[dataset] Skipped empty/unreadable file: S50A06T01.csv
[dataset] Skipped empty/unreadable file: S60A11T01.csv
[dataset] Skipped empty/unreadable file: S60A02T02.csv
[dataset] Skipped empty/unreadable file: S50A12T03.csv
[dataset] Skipped empty/unreadable file: S52A03T01.csv
[dataset] Skipped empty/unreadable file: S50A05T02.csv
[dataset] Skipped empty/unreadable file: S50A01T02.csv
[dataset] Skipped empty/unreadable file: S52A05T03.csv
[dataset] Skipped empty/unreadable file: S50A11T03.csv
[dataset] Skipped empty/unreadable file: S50A02T05.csv
[dataset] Skipped empty/unreadable file: S50A01T03.csv
[dataset] Skipped empty/unreadable file: S50A02T04.csv
[dataset] Skipped empty/unreadable file: S56A10T02.csv
[dataset] Skipped empty/unreadable file: S52A05T05.csv
[dataset] Skipped empty/unreadable file: 

In [ ]:
# !python train.py \
#   --stage 3 \
#   --stage1_ckpt checkpoints/stage1_ae/stage1/best.pt \
#   --stage2_ckpt checkpoints/stage2_imu2latent/stage2/best.pt \
#   --skeleton_folder /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton \
#   --sensor1_folder /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_wrist \
#   --sensor2_folder /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_hip \
#   --run_name stage3_cond \
#   --epochs 50 --batch_size 16 --lr 1e-4 \
#   --stage3_use_imu --stage3_cond joint \
#   --num_classes 2 --cls_weight 1.0 \
#     --align_mode truncate_min

2026-02-28 05:44:41,287 | INFO | train | Run dir: checkpoints/stage3_cond
2026-02-28 05:44:41,287 | INFO | train | Device: cuda
[dataset] Skipped empty/unreadable file: S51A13T02.csv
[dataset] Skipped empty/unreadable file: S50A06T01.csv
[dataset] Skipped empty/unreadable file: S60A11T01.csv
[dataset] Skipped empty/unreadable file: S60A02T02.csv
[dataset] Skipped empty/unreadable file: S50A12T03.csv
[dataset] Skipped empty/unreadable file: S52A03T01.csv
[dataset] Skipped empty/unreadable file: S50A05T02.csv
[dataset] Skipped empty/unreadable file: S50A01T02.csv
[dataset] Skipped empty/unreadable file: S52A05T03.csv
[dataset] Skipped empty/unreadable file: S50A11T03.csv
[dataset] Skipped empty/unreadable file: S50A02T05.csv
[dataset] Skipped empty/unreadable file: S50A01T03.csv
[dataset] Skipped empty/unreadable file: S50A02T04.csv
[dataset] Skipped empty/unreadable file: S56A10T02.csv
[dataset] Skipped empty/unreadable file: S52A05T05.csv
[dataset] Skipped empty/unreadable file: S52A05

In [7]:
!python generate.py \
  --stage3_ckpt /home/qsw26/smartfall/gait_loss/latent_diff_skeleton/checkpoints/stage3_cond/stage3/best.pt\
  --sensor_ckpt /home/qsw26/smartfall/gait_loss/latent_diff_skeleton/checkpoints/stage2_imu2latent/stage2/best.pt \
  --skeleton_folder /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/skeleton \
  --sensor_folder1 /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_wrist \
  --sensor_folder2 /home/qsw26/smartfall/gait_loss/SmartFallMM-Dataset/young/accelerometer/meta_hip \
  --window_size 60 \
  --window_stride 30 \
  --align_mode truncate_min \
  --num_classes 2 \
  --latent_dim 256 \
  --num_joints 32 \
  --diffusion_steps 500 \
  --imu_hidden_dim 256 \
  --imu_norm zscore \
  --out_gif latent_diff_skeleton/gif_tl/generated_aligned.gif



================ CHECKPOINT LOAD ================
Path:   /home/qsw26/smartfall/gait_loss/latent_diff_skeleton/checkpoints/stage2_imu2latent/stage2/best.pt
Strict: False
Missing keys:    0
Unexpected keys: 0


================ CHECKPOINT LOAD ================
Path:   /home/qsw26/smartfall/gait_loss/latent_diff_skeleton/checkpoints/stage3_cond/stage3/best.pt
Strict: False
Missing keys:    0
Unexpected keys: 0

Generating samples based on sensor inputs...
[dataset] Skipped empty/unreadable file: S51A13T02.csv
[dataset] Skipped empty/unreadable file: S50A06T01.csv
[dataset] Skipped empty/unreadable file: S60A11T01.csv
[dataset] Skipped empty/unreadable file: S60A02T02.csv
[dataset] Skipped empty/unreadable file: S50A12T03.csv
[dataset] Skipped empty/unreadable file: S52A03T01.csv
[dataset] Skipped empty/unreadable file: S50A05T02.csv
[dataset] Skipped empty/unreadable file: S50A01T02.csv
[dataset] Skipped empty/unreadable file: S52A05T03.csv
[dataset] Skipped empty/unreadable file: S50A1